In [10]:
### Data Augmentation makes underfit but best for overfitting

In [11]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

--2023-10-05 21:42:30--  https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.207, 142.251.162.207, 173.194.215.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip.1’

cats_and_dogs_filte 100%[===================>]  65.43M   173MB/s    in 0.4s    

2023-10-05 21:42:31 (173 MB/s) - ‘cats_and_dogs_filtered.zip.1’ saved [68606236/68606236]



In [13]:
# Extract the archive
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Assign training and validation set directories
base_dir = 'tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [14]:
# This code has changed. Now instead of the ImageGenerator just rescaling
# the image, we also rotate and do other operations
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [15]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), activation = "relu", input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
    ]
)

model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])


class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') is not None and logs.get('val_accuracy') > 0.70:
            print("\nReached 70% val accuracy so cancelling training!")
            self.model.stop_training = True

In [17]:
history_1 = model_1.fit(train_generator,
                        epochs = 100,
                        validation_data=validation_generator,
                        callbacks = [myCallback()])

Epoch 1/100
100/100 [==============================] - 75s 728ms/step - loss: 0.6983 - accuracy: 0.5035 - val_loss: 0.6933 - val_accuracy: 0.5030
Epoch 2/100
100/100 [==============================] - 69s 693ms/step - loss: 0.6841 - accuracy: 0.5625 - val_loss: 0.6856 - val_accuracy: 0.5000
Epoch 3/100
100/100 [==============================] - 67s 668ms/step - loss: 0.6677 - accuracy: 0.5835 - val_loss: 0.6339 - val_accuracy: 0.6570
Epoch 4/100
100/100 [==============================] - 67s 671ms/step - loss: 0.6634 - accuracy: 0.6120 - val_loss: 0.6379 - val_accuracy: 0.6660
Epoch 5/100
100/100 [==============================] - 67s 671ms/step - loss: 0.6397 - accuracy: 0.6145 - val_loss: 0.6277 - val_accuracy: 0.6540
Epoch 6/100
100/100 [==============================] - 66s 664ms/step - loss: 0.6301 - accuracy: 0.6450 - val_loss: 0.6354 - val_accuracy: 0.6190
Epoch 7/100
100/100 [==============================] - 68s 683ms/step - loss: 0.6297 - accuracy: 0.6490 - val_loss: 0.5789 -